# Camera Calibration

In this tutorial,
- We will learn about distortions in camera, intrinsic and extrinsic parameters of camera etc.
- We will learn to find these parameters, undistort images etc.

In [ ]:
import numpy as np
import cv2 # OpenCV-Python
%matplotlib inline
import matplotlib.pyplot as plt
# This module is for search filesystem with pattern matching
import glob

## `glob` - Unix style pathname pattern expansion

- [Official document](https://docs.python.org/2/library/glob.html)
- [한글 python wiki](https://wikidocs.net/83)

In [ ]:
print glob.glob('./*')

In [ ]:
print glob.glob('./')

In [ ]:
print glob.glob('../*')

In [ ]:
print glob.glob('images/*')

In [ ]:
print glob.glob('images/left*.jpg')

## 1. Find chessboard patterns

Prepare 3D points from real world space in (X, Y, Z).

For simplicity, we can say chessboard was kept stationary at XY plane, (so Z=0 always)

So, we make points like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)

But if we know the square size, e.g. 30mm, we can pass the values as (0,0,0), (30, 0, 0), (60,  0, 0) ...

In [ ]:
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
print objp

Make arrays to store object points and image points from all the images

In [ ]:
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

Get image filename lists by using `glob.g1ob`

In [ ]:
images = glob.glob('images/left*.jpg')
print images

Let's find chessboard pattern in one image

In [ ]:
fname = images[0]
img = cv2.imread(fname)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
plt.imshow(gray, cmap='gray')

Find the chess board corners.

Corners are (x, y) coordinate of image.

In [ ]:
ret, corners = cv2.findChessboardCorners(gray, (7,6),None)
print ret
print corners

If found, add object points, image points (after refining them)

And draw chessboard corners

In [ ]:
if ret == True:
    objpoints.append(objp)

    # Termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # Refine corners
    before_refined = corners.copy()
    cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
    imgpoints.append(corners)

    # Draw and display the corners
    cv2.drawChessboardCorners(img, (7,6), corners,ret)
    plt.figure()
    plt.title(fname)
    plt.imshow(img)

`criteria` provides criteria for termination of the iterative process of corner refinement.

It must be formatted as `(Termination type, Max count, Epsilon)`

`Termination type` can be one of
- `cv2.TERM_CRITERIA_EPS`
- `cv2.TERM_CRITERIA_MAX_ITER`
- `cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER`.

In [ ]:
print criteria

Let's check how much corenr points are refined

In [ ]:
for before_refined_corner, after_refined_corner in zip(before_refined, corners):
    print "Before: ", before_refined_corner, "After: ", after_refined_corner

Check which values are saved in `objpoints` and `imgpoints`

In [ ]:
for objpoint, imgpoint in zip(objpoints[0], imgpoints[0]):
    print "objpoint: ", objpoint, "imgpoint: ", imgpoint

Now, lets find and draw chessboard corners in all images

In [ ]:
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('images/left*.jpg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        
        # Termination criteria
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        
        # Refine corners
        cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,6), corners,ret)
        plt.figure()
        plt.title(fname)
        plt.imshow(img)

## 2. Calibration
Find camera matrix, distortion coefficients, rotation and translation vectors

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [ ]:
print gray.shape
print gray.shape[::-1]

In [ ]:
print ret

In [ ]:
print mtx

In [ ]:
print dist

In [ ]:
for rvec, tvec in zip(rvecs, tvecs):
    print "rvec: ", rvec.T, "tvec: ", tvec.T

## 3. Undistortion

Before undistortion, let's refine camera matrix with distortion coefficients.

In [ ]:
img = cv2.imread('images/left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

In [ ]:
print mtx
print newcameramtx
print roi

Now, undistort whole image with `newcameramtx` and `dist`

In [ ]:
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.title("Before undistortion")
plt.imshow(img)
plt.subplot(1,2,2)
plt.title("After undistortion")
plt.imshow(dst)

## 4. Calculate re-projection error

Re-projection error gives a good estimation of just how exact is the found parameters.
This should be as close to zero as possible.

In [ ]:
mean_error = 0
for i in xrange(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error

print "total error: ", mean_error/len(objpoints)

---
### Reference

Please see the following official tutorials for more detailed explanation.

- [Camera Calibration - OpenCV documentation](http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_calib3d/py_calibration/py_calibration.html#calibration)